In [2]:
import requests 
import pandas as pd 
import numpy as np 

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

from IPython.display import Image 
from IPython.core.display import HTML 
    
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium 

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [120]:
ClIENT_ID = 'VE2GTXXMG3JNTA0OTI0BEY0G2ZFXVMIOYB1VDZKBWFYB3OZR' # your Foursquare ID
ClIENT_SECRET = 'EIFGAJKIYQAVCUPVV2VZVRV3NF4HTOAVZDHLCJDHWHUNMQET' # your Foursquare Secret
VERSION = '20180604'
LIMIT =30
print('Your credentails:')
print('Foursquare_ID: ' + ClIENT_ID)
print('Foursquare_Secret:' + ClIENT_SECRET)

Your credentails:
Foursquare_ID: VE2GTXXMG3JNTA0OTI0BEY0G2ZFXVMIOYB1VDZKBWFYB3OZR
Foursquare_Secret:EIFGAJKIYQAVCUPVV2VZVRV3NF4HTOAVZDHLCJDHWHUNMQET


In [121]:
city = 'Chongquing'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(city)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

30.6668771 104.0954336


In [139]:
search_query = 'Hotel'
radius = 1000

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=VE2GTXXMG3JNTA0OTI0BEY0G2ZFXVMIOYB1VDZKBWFYB3OZR&client_secret=EIFGAJKIYQAVCUPVV2VZVRV3NF4HTOAVZDHLCJDHWHUNMQET&ll=30.6668771,104.0954336&v=20180604&query=Hotel&radius=1000&limit=30'

In [140]:
results = requests.get(url).json()

In [147]:
venues = results['response']['venues']
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,4bd7aabf304fce72d7da33ab,No.55 Jianshe Road,CN,成都市,中国,435,"[No.55 Jianshe Road, 成都市, 四川, 中国]","[{'label': 'display', 'lat': 30.667997, 'lng':...",30.667997,104.099796,四川,Hualian Hotel Chengdu,v-1578803137
1,"[{'id': '4bf58dd8d48988d130941735', 'name': 'B...",False,51bfc476498e6a177144771f,NaN,CN,NaN,中国,467,[中国],"[{'label': 'display', 'lat': 30.67074640335947...",30.670746,104.097339,NaN,Jin zhu yu sha hotel Chendu,v-1578803137
2,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,4bd1a5405e0cce72cfc5a184,No 10 Xinghui East Road,CN,成都市,中国,971,"[No 10 Xinghui East Road, 成都市, 四川, 中国]","[{'label': 'display', 'lat': 30.672068, 'lng':...",30.672068,104.087283,四川,Tu Ha Hotel Chengdu,v-1578803137
3,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,52820ffd498e95ebf3b0c521,NaN,CN,NaN,中国,1159,[中国],"[{'label': 'display', 'lat': 30.657236, 'lng':...",30.657236,104.100015,NaN,五粮液大酒店Wuliangye Grand Hotel فندق وو ليانغ يه,v-1578803137
4,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,536ea9b4498effee049a3518,NaN,CN,成都市,中国,921,"[成都市, 四川, 中国]","[{'label': 'display', 'lat': 30.65994824998057...",30.659948,104.090159,四川,ShangJinHotel,v-1578803137


In [148]:
clean_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')]+ ['id']
clean_dataframe = dataframe.loc[:,clean_columns]

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

clean_dataframe['categories'] = clean_dataframe.apply(get_category_type, axis=1)

clean_dataframe.columns = [column.split('.')[-1] for column in clean_dataframe.columns]

clean_dataframe.head()

,name,categories,address,cc,city,country,distance,formattedAddress,labeledLatLngs,lat,lng,state,id
0,Hualian Hotel Chengdu,Hotel,No.55 Jianshe Road,CN,成都市,中国,435,"[No.55 Jianshe Road, 成都市, 四川, 中国]","[{'label': 'display', 'lat': 30.667997, 'lng':...",30.667997,104.099796,四川,4bd7aabf304fce72d7da33ab
1,Jin zhu yu sha hotel Chendu,Building,NaN,CN,NaN,中国,467,[中国],"[{'label': 'display', 'lat': 30.67074640335947...",30.670746,104.097339,NaN,51bfc476498e6a177144771f
2,Tu Ha Hotel Chengdu,Hotel,No 10 Xinghui East Road,CN,成都市,中国,971,"[No 10 Xinghui East Road, 成都市, 四川, 中国]","[{'label': 'display', 'lat': 30.672068, 'lng':...",30.672068,104.087283,四川,4bd1a5405e0cce72cfc5a184
3,五粮液大酒店Wuliangye Grand Hotel فندق وو ليانغ يه,Hotel,NaN,CN,NaN,中国,1159,[中国],"[{'label': 'display', 'lat': 30.657236, 'lng':...",30.657236,104.100015,NaN,52820ffd498e95ebf3b0c521
4,ShangJinHotel,Hotel,NaN,CN,成都市,中国,921,"[成都市, 四川, 中国]","[{'label': 'display', 'lat': 30.65994824998057...",30.659948,104.090159,四川,536ea9b4498effee049a3518


In [149]:
clean_dataframe2= clean_dataframe.drop(['cc', 'city', 'country', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'id'], axis=1)
clean_dataframe2

,name,categories,address,lat,lng,state
0,Hualian Hotel Chengdu,Hotel,No.55 Jianshe Road,30.667997,104.099796,四川
1,Jin zhu yu sha hotel Chendu,Building,NaN,30.670746,104.097339,NaN
2,Tu Ha Hotel Chengdu,Hotel,No 10 Xinghui East Road,30.672068,104.087283,四川
3,五粮液大酒店Wuliangye Grand Hotel فندق وو ليانغ يه,Hotel,NaN,30.657236,104.100015,NaN
4,ShangJinHotel,Hotel,NaN,30.659948,104.090159,四川


In [150]:
array= ['Hotel']
hotel_dataframe= clean_dataframe2.loc[clean_dataframe2['categories'].isin(array)]
hotel_dataframe

,name,categories,address,lat,lng,state
0,Hualian Hotel Chengdu,Hotel,No.55 Jianshe Road,30.667997,104.099796,四川
2,Tu Ha Hotel Chengdu,Hotel,No 10 Xinghui East Road,30.672068,104.087283,四川
3,五粮液大酒店Wuliangye Grand Hotel فندق وو ليانغ يه,Hotel,NaN,30.657236,104.100015,NaN
4,ShangJinHotel,Hotel,NaN,30.659948,104.090159,四川


In [151]:
hotel_neighbourhood_df = hotel_dataframe.drop_duplicates(subset='name', keep="first")
hotel_neighbourhood_df

,name,categories,address,lat,lng,state
0,Hualian Hotel Chengdu,Hotel,No.55 Jianshe Road,30.667997,104.099796,四川
2,Tu Ha Hotel Chengdu,Hotel,No 10 Xinghui East Road,30.672068,104.087283,四川
3,五粮液大酒店Wuliangye Grand Hotel فندق وو ليانغ يه,Hotel,NaN,30.657236,104.100015,NaN
4,ShangJinHotel,Hotel,NaN,30.659948,104.090159,四川


In [152]:
hotel_map = folium.Map(location=[latitude, longitude], zoom_start=14)

for lat, lng, name, categories, address in zip(hotel_neighbourhood_df['lat'], hotel_neighbourhood_df['lng'], 
                                           hotel_neighbourhood_df['name'], hotel_neighbourhood_df['categories'],\
                                               hotel_neighbourhood_df['address']):
    label = '{}, {}'.format(name, address)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(hotel_map)  
    
hotel_map